In [1]:
import numpy as np
import random

In [70]:
def read_qap_instance(filepath, elements_per_row=20):
    with open(filepath, 'r') as f:
        n = int(f.readline().strip())

        A = []
        B = []

        f.readline()
        for _ in range(n):
            row_data = []
            while len(row_data) < n:
                row_data += [int(x) for x in f.readline().strip().split()][:elements_per_row] 
            A.append(row_data[:n])

        f.readline()
        for _ in range(n):
            row_data = []
            while len(row_data) < n:
                row_data += [int(x) for x in f.readline().strip().split()][:elements_per_row] 
            B.append(row_data[:n])

        A = np.array(A)
        B = np.array(B)

        return n, A, B

In [71]:
def construct_initial_solution(n):
    """Constructs a random initial solution (permutation)."""
    permutation = list(range(n))
    random.shuffle(permutation)

    return np.array(permutation)

def get_parameters():
    """Gets tabu size and maximum iterations from the user."""
    while True:
        try:
            tabu_size = int(input("Enter tabu size: "))
            max_iter = int(input("Enter maximum iterations: "))
            if tabu_size > 0 and max_iter > 0:
                return tabu_size, max_iter
            else:
                print("Please enter positive values.")
        except ValueError:
            print("Invalid input. Please enter integers.")

def explore_moves(solution, F, D, tabu_list):
    """Explores all pairwise moves and returns the best allowed move."""
    best_move = None
    best_move_cost = float('inf')

    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            if (i, j) not in tabu_list and (j, i) not in tabu_list:
                new_solution = solution.copy()
                new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
                cost = calculate_cost(new_solution, F, D)

                if cost < best_move_cost:
                    best_move = (i, j)
                    best_move_cost = cost
    
    return best_move, best_move_cost

def calculate_cost(solution, F, D):
    """Calculates the cost of a given solution."""
    cost = 0
    for i in range(len(solution)):
        for j in range(len(solution)):
            cost += F[i, j] * D[solution[i], solution[j]]
            #cost += D[i, j] * F[solution[i], solution[j]]

    return cost

def get_user_choice():
    """Gets user input for restart, best solution, or LTM."""
    while True:
        print("\nChoose an option:")
        print("1. Restart with a new initial solution")
        print("2. Use the best solution found so far")
        print("3. Apply long-term memory (LTM)")
        print("4. Stop")

        try:
            choice = int(input("Enter your choice: "))
            if choice in [1, 2, 3 ,4]:
                return choice
            else:
                print("Invalid choice. Please enter 1, 2, 3 or 4.")
        except ValueError:
            print("Invalid input. Please enter an integer.")

def tabu_navigation(F, D, ltm_penalty=1):
    """
    Tabu Navigation algorithm for QAP
    """
    n = len(F)  # number of facilities/locations

    # initialize long-term memory (LTM)
    LTM = np.zeros((n, n))

    best_solution = None
    best_cost = float('inf')
    choice = 0

    while True:
        # construction phase - init permutation - random
        if choice == 0 or choice == 3:
            solution = construct_initial_solution(n)

        # improvement phase - pairwise moves
        tabu_size, max_iter = get_parameters()
        tabu_list = []

        for k in range(max_iter):
            # explore all pairwise moves
            best_move, best_move_cost = explore_moves(solution, F, D, tabu_list)

            # perform the best admissible move
            solution[best_move[0]], solution[best_move[1]] = solution[best_move[1]], solution[best_move[0]]
            tabu_list.append(best_move)

            # update tabu list (remove oldest entries if needed)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)

            # update long-term memory (LTM)
            LTM[best_move[0], best_move[1]] += 1
            LTM[best_move[1], best_move[0]] += 1

            # update best solution if found
            current_cost = calculate_cost(solution, F, D)
            if current_cost < best_cost:
                best_cost = current_cost
                best_solution = solution.copy()
            
            if max_iter < 50:
                print(f"{k} iter: best_cost = {best_cost}")
            elif k % 10 == 0:
                print(f"{k} iter: best_cost = {best_cost}")

        # restart or apply LTM
        choice = get_user_choice()
        if choice == 1:
            continue  # restart with new init solution
        elif choice == 2:
            solution = best_solution.copy()  # use the best solution
        elif choice == 3:
            D = D + ltm_penalty * LTM
            solution = construct_initial_solution(n)  # restart with modified D
        else:
            break

    return best_solution, best_cost

In [76]:
n, flow_mat, dist_mat = read_qap_instance("../instances/nug15.dat", elements_per_row=30) # for sko: elements_per_row=20
print("Size:", n)
print("Flow Matrix:", flow_mat.shape)
print("Distance Matrix:", dist_mat.shape)

Size: 15
Flow Matrix: (15, 15)
Distance Matrix: (15, 15)


In [77]:
best_solution, best_cost = tabu_navigation(flow_mat, dist_mat)
print("Best Solution:", best_solution)
print("Cost:", best_cost)

0 iter: best_cost = 1486
1 iter: best_cost = 1394
2 iter: best_cost = 1302
3 iter: best_cost = 1258
4 iter: best_cost = 1236
5 iter: best_cost = 1216
6 iter: best_cost = 1204
7 iter: best_cost = 1204
8 iter: best_cost = 1204
9 iter: best_cost = 1204

Choose an option:
1. Restart with a new initial solution
2. Use the best solution found so far
3. Apply long-term memory (LTM)
4. Stop
0 iter: best_cost = 1204
1 iter: best_cost = 1204
2 iter: best_cost = 1204
3 iter: best_cost = 1204
4 iter: best_cost = 1204
5 iter: best_cost = 1204
6 iter: best_cost = 1204
7 iter: best_cost = 1204
8 iter: best_cost = 1204
9 iter: best_cost = 1204

Choose an option:
1. Restart with a new initial solution
2. Use the best solution found so far
3. Apply long-term memory (LTM)
4. Stop
Best Solution: [ 8 12  4 13 14  7  1  6  2  5 10  0 11  3  9]
Cost: 1204
